In [58]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score

# Loading Dataset

In [59]:
train_df = pd.read_csv(r"N:\Encryptix Intermship\Task 3\fraudTrain.csv")
test_df = pd.read_csv(r"N:\Encryptix Intermship\Task 3\fraudTest.csv")
print(len(train_df), len(test_df))

1296675 555719


In [60]:
train_df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [61]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

# Data Preprocessing

In [50]:
print("Checking Duplicate values:",train_df.duplicated().sum(), test_df.duplicated().sum())
print("Checking Null values:", train_df.isna().sum().sum(), test_df.isna().sum().sum())

Checking Duplicate values: 0 0
Checking Null values: 0 0


In [51]:
from sklearn.preprocessing import LabelEncoder
le_gender = LabelEncoder()
le_category = LabelEncoder()
le_city = LabelEncoder()
le_job = LabelEncoder()

# Combine training and test data for encoding
combined_df = pd.concat([train_df, test_df])

# Fit the encoders on the combined data
le_gender.fit(combined_df['gender'])
le_category.fit(combined_df['category'])
le_city.fit(combined_df['city'])
le_job.fit(combined_df['job'])

# Preprocessing for training data
train_df['gender'] = le_gender.transform(train_df['gender'])
train_df['category'] = le_category.transform(train_df['category'])
train_df['city'] = le_city.transform(train_df['city'])
train_df['job'] = le_job.transform(train_df['job'])

# Preprocessing for test data
test_df['gender'] = le_gender.transform(test_df['gender'])
test_df['category'] = le_category.transform(test_df['category'])
test_df['city'] = le_city.transform(test_df['city'])
test_df['job'] = le_job.transform(test_df['job'])


In [52]:
train_data.head(10), test_data.head(10)

(  gender category     amt city  job  is_fraud
 0      1        8    4.97  526  370         0
 1      1        4  107.23  612  428         0
 2      0        0  220.11  468  307         0
 3      0        2   45.00   84  328         0
 4      0        9   41.96  216  116         0
 5      1        2   94.63  223  479         0
 6      1        3   44.54  351   29         0
 7      0        2   71.65  236  127         0
 8      1        9    4.27  474  375         0
 9      1        4  198.39  149  329         0,
   gender category     amt city  job  is_fraud
 0      0       10    2.86  157  275         0
 1      1       10   29.84   16  392         0
 2      1        5   41.28   61  259         0
 3      0        9   60.05  764  407         0
 4      0       13    3.19  247  196         0
 5      1        7   19.55   90  361         0
 6      1        5  133.93  117  455         0
 7      1       10   10.37  725  124         0
 8      0       12    4.37  503   13         0
 9      1   

In [53]:
X_train = train_data[['gender', "category", "amt", "city", "job"]]
Y_train = train_data["is_fraud"]

X_test = test_data[['gender', "category", "amt", "city", "job"]]
Y_test = test_data["is_fraud"]

# Train the model (Logistic Regression)

In [54]:
lr_model = LogisticRegression(penalty="l1", solver="liblinear")
lr_model.fit(X_train, Y_train)

LogisticRegression(penalty='l1', solver='liblinear')

In [55]:
Y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("Testing Accuracy Score:", accuracy)

Testing Accuracy Score: 0.9955013235106231


# Testing the Model with custom data

In [57]:
# Custom input data
custom_data = {
    'gender': 'F',
    'category': 'grocery_pos',
    'amt': 100.0,
    'city': 'Grenada',
    'job': 'Event organiser'
}

# Preprocess the custom input data
custom_data['gender'] = le_gender.transform([custom_data['gender']])[0]
custom_data['category'] = le_category.transform([custom_data['category']])[0]
custom_data['city'] = le_city.transform([custom_data['city']])[0]
custom_data['job'] = le_job.transform([custom_data['job']])[0]

# Convert to DataFrame for model prediction
custom_input_df = pd.DataFrame([custom_data])

# Make prediction
custom_prediction = lr_model.predict(custom_input_df)

# Print the prediction
print("Custom Input Prediction (1 for fraud, 0 for not fraud):", custom_prediction[0])

Custom Input Prediction (1 for fraud, 0 for not fraud): 0
